# 层和块

首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.2415, -0.1362, -0.0434,  0.1452,  0.0516, -0.0029, -0.0849, -0.1138,
          0.1555, -0.0411],
        [-0.4121, -0.0884, -0.0022,  0.1045, -0.0567, -0.0720, -0.0239, -0.1836,
          0.1728,  0.0176]], grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.0408, -0.0039, -0.1304, -0.1900, -0.0659,  0.1552, -0.0697,  0.0617,
         -0.1475, -0.3214],
        [ 0.1596,  0.0915, -0.2875, -0.2926,  0.0018,  0.0994, -0.0799,  0.0995,
         -0.1499, -0.2732]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0635,  0.0091, -0.1554,  0.0962, -0.0199,  0.2631, -0.0582, -0.1048,
          0.2033, -0.0445],
        [-0.0120,  0.0334, -0.0129,  0.0403,  0.1250,  0.2563, -0.1767, -0.0914,
          0.2401, -0.1676]], grad_fn=<AddmmBackward>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.2615, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2314, grad_fn=<SumBackward0>)